In [9]:
from pathlib import Path
import pandas as pd
import yaml
from tqdm import tqdm
import cv2
import shutil
import numpy as np
np.random.seed(42)

class_name_to_id = {
    "adj" : "0",
    "int": "1",
    "geo": "2",
    "pro": "3",
    "non": "4"
}

In [10]:
class_name = "adj"
dataset_name = f"data_{class_name}"
source_dataset_dir = Path('./dataset')
classes_path = source_dataset_dir / 'classes.txt'
images_dir = Path(f'{dataset_name}/images')

labels_dir = Path(f'{dataset_name}/labels')
(images_dir / "train").mkdir(exist_ok=True, parents=True)
(images_dir / "val").mkdir(exist_ok=True, parents=True)
(labels_dir / "train").mkdir(exist_ok=True, parents=True)
(labels_dir / "val").mkdir(exist_ok=True, parents=True)


In [11]:
def get_only_one_class(class_name: str, txt_file: Path):
    annotation = open(txt_file).read()
    new_annotation = []
    for i, line in enumerate(annotation.splitlines()):
        class_id, *points = line.split()
        if class_name_to_id[class_name] == class_id:
            new_annotation.append('0' + line[1:])
    return new_annotation

In [12]:
for txt_file in tqdm(source_dataset_dir.glob('*.txt')):
    if "classes" in txt_file.stem:
        continue
    image_file = txt_file.with_suffix(".jpg")
    class_data = get_only_one_class(class_name, txt_file)
    if len(class_data) > 0:
        subfolder = Path("train") if np.random.rand() < 0.9 else Path("val")
        shutil.copy(txt_file, labels_dir / subfolder / txt_file.name)
        with open(labels_dir / subfolder / txt_file.name, 'wt') as fil:
            fil.write("\n".join(class_data))
        shutil.copy(image_file, images_dir / subfolder / image_file.name)

1163it [00:00, 1898.48it/s]


In [13]:
import yaml
result_yaml = {
    "path": f"/home/simba9/pet/shov/{dataset_name}",
    "train": "images/train",
    "val": "images/val"
}
with open(classes_path) as f:
    names = []
    for line in f:
        names.append(line.strip())
    
    result_yaml["names"] = {
        0: class_name
    }
with open(Path(f'./shov_{class_name}.yml'), 'w') as yaml_file:
    yaml.dump(result_yaml, yaml_file, default_flow_style=False)
